![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use AutoAI RAG and Chroma to create a pattern and get information from `ibm-watsonx-ai` SDK documentation

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate the usage of IBM AutoAI RAG. The AutoAI RAG experiment conducted in this notebook uses data scraped from the `ibm-watsonx-ai` SDK documentation.

Some familiarity with Python is helpful. This notebook uses Python 3.11.


## Learning goal

The learning goals of this notebook are:

- Create an AutoAI RAG job that will find the best RAG pattern based on provided data


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [RAG Optimizer definition](#definition)
- [RAG Experiment run](#run)
- [RAG Patterns comparison and testing](#comparison)
- [Historical runs](#runs)
- [Clean up](#cleanup)
- [Summary and next steps](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the required modules and dependencies

In [ ]:
!pip install -U 'ibm-watsonx-ai[rag]>=1.3.26' | tail -n 1
!pip install -U "langchain_community>=0.3,<0.4" | tail -n 1

### Defining the watsonx.ai credentials
This cell defines the credentials required to work with the watsonx.ai Runtime service.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [ ]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your watsonx.ai api key (hit enter): "),
)

### Defining the project id
The foundation model requires a project id that provides the context for the call. We will try to obtain the id directly from the project in which this notebook runs. If this fails, you'll have to provide the project id.

In [2]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

Create an instance of APIClient with authentication details.

In [3]:
from ibm_watsonx_ai import APIClient

client = APIClient(credentials=credentials, project_id=project_id)

<a id="definition"></a>

## RAG Optimizer definition

### Defining a connection to training data

Upload training data to a COS bucket and then define a connection to this file. This example uses the `Base` description from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/fm_model_inference.html) documentation.

The code in the next cell uploads training data to the bucket.

In [4]:
import os
import requests

url = "https://ibm.github.io/watsonx-ai-python-sdk/base.html"

document_filename = "base.html"

response = requests.get(url)

response.raise_for_status()

if not os.path.isfile(document_filename):
    with open(document_filename, "w", encoding="utf-8") as file:
        file.write(response.text)

document_asset_details = client.data_assets.create(name=document_filename, file_path=document_filename)

document_asset_id = client.data_assets.get_id(document_asset_details)
document_asset_id

Creating data asset...
SUCCESS


'3735c60e-6ead-40a1-a73a-aa7f02ba850d'

Define a connection to training data.

In [5]:
from ibm_watsonx_ai.helpers import DataConnection

input_data_references = [DataConnection(data_asset_id=document_asset_id)]

### Defining a connection to test data

Upload a `json` file that will be used for benchmarking to COS and then define a connection to this file. This example uses content from the [`ibm_watsonx_ai`](https://ibm.github.io/watsonx-ai-python-sdk/index.html) SDK documentation.

In [6]:
benchmarking_data_IBM_page_content = [
    {
        "question": "How can you set or refresh user request headers using the APIClient class?",
        "correct_answer": "client.set_headers({'Authorization': 'Bearer <token>'})",
        "correct_answer_document_ids": [
            "base.html"
        ]
    },
    {
        "question": "How to initialise Credentials object with api_key",
        "correct_answer": "credentials = Credentials(url = 'https://us-south.ml.cloud.ibm.com', api_key = '***********')",
        "correct_answer_document_ids": [
            "base.html"
        ]
    }
]

The code in the next cell uploads testing data to the bucket as a `json` file.

In [7]:
import json

test_filename = "benchmarking_data_Base.json"

if not os.path.isfile(test_filename):
    with open(test_filename, "w") as json_file:
        json.dump(benchmarking_data_IBM_page_content, json_file, indent=4)

test_asset_details = client.data_assets.create(name=test_filename, file_path=test_filename)

test_asset_id = client.data_assets.get_id(test_asset_details)
test_asset_id

Creating data asset...
SUCCESS


'6eeb3620-8933-4592-8b5b-0d19ac6f2133'

Define connection information to testing data.

In [8]:
test_data_references = [DataConnection(data_asset_id=test_asset_id)]

### RAG Optimizer configuration

Provide the input information for AutoAI RAG optimizer:
- `name` - experiment name
- `description` - experiment description
- `max_number_of_rag_patterns` - maximum number of RAG patterns to create
- `optimization_metrics` - target optimization metrics

In [12]:
from ibm_watsonx_ai.experiment import AutoAI

experiment = AutoAI(credentials, project_id=project_id)

rag_optimizer = experiment.rag_optimizer(
    name='AutoAI RAG run - Base documentation',
    description="AutoAI RAG Optimizer on ibm_watsonx_ai Base documentation",
    foundation_models=["mistralai/mistral-small-3-1-24b-instruct-2503"],
    embedding_models=["ibm/slate-125m-english-rtrvr"],
    retrieval_methods=["simple"],
    chunking=[
        {
            "chunk_size": 512,
            "chunk_overlap": 64,
            "method": "recursive"
        }
    ],
    max_number_of_rag_patterns=4,
    optimization_metrics=[AutoAI.RAGMetrics.ANSWER_CORRECTNESS]
)

Configuration parameters can be retrieved via `get_params()`.

In [13]:
rag_optimizer.get_params()

{'name': 'AutoAI RAG run - Base documentation',
 'description': 'AutoAI RAG Optimizer on ibm_watsonx_ai Base documentation',
 'chunking': [{'chunk_size': 512, 'chunk_overlap': 64, 'method': 'recursive'}],
 'embedding_models': ['ibm/slate-125m-english-rtrvr'],
 'retrieval_methods': ['simple'],
 'generation': {'foundation_models': [{'model_id': 'mistralai/mistral-small-3-1-24b-instruct-2503'}]},
 'max_number_of_rag_patterns': 4,
 'optimization_metrics': ['answer_correctness']}

<a id="run"></a>
## RAG Experiment run

Call the `run()` method to trigger the AutoAI RAG experiment. You can either use interactive mode (synchronous job) or background mode (asynchronous job) by specifying `background_mode=True`.

In [14]:
run_details = rag_optimizer.run(
    input_data_references=input_data_references,
    test_data_references=test_data_references,
    background_mode=False
)



##############################################

Running 'b4b57086-916a-4228-af2d-e420159bcf08'

##############################################


pending.............
running......
completed
Training of 'b4b57086-916a-4228-af2d-e420159bcf08' finished successfully.


You can use the `get_run_status()` method to monitor AutoAI RAG jobs in background mode.

In [15]:
rag_optimizer.get_run_status()

'completed'

<a id="comparison"></a>
## Comparison and testing of RAG Patterns

You can list the trained patterns and information on evaluation metrics in the form of a Pandas DataFrame by calling the `summary()` method. You can use the DataFrame to compare all discovered patterns and select the one you like for further testing.

In [16]:
summary = rag_optimizer.summary()
summary

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,retrieval.hybrid_ranker,generation.model_id
Pattern_Name,,,,,,,,,,,,
Pattern2,0.7083,0.2926,1.0,recursive,512,64,ibm/slate-125m-english-rtrvr,cosine,simple,5,,mistralai/mistral-small-3-1-24b-instruct-2503
Pattern1,0.4583,0.2808,1.0,recursive,512,64,ibm/slate-125m-english-rtrvr,cosine,simple,3,,mistralai/mistral-small-3-1-24b-instruct-2503


Additionally, you can pass the `scoring` parameter to the summary method, to filter RAG patterns starting with the best.

```python
summary = rag_optimizer.summary(scoring="faithfulness")
```

In [17]:
rag_optimizer.get_run_details()

{'entity': {'hardware_spec': {'id': 'a6c4923b-b8e4-444c-9f43-8a7ec3020110',
   'name': 'L'},
  'input_data_references': [{'location': {'href': '/v2/assets/3735c60e-6ead-40a1-a73a-aa7f02ba850d?project_id=eac8bfe2-a00b-43ca-846b-305af5cc6395',
     'id': '3735c60e-6ead-40a1-a73a-aa7f02ba850d'},
    'type': 'data_asset'}],
  'parameters': {'constraints': {'chunking': [{'chunk_overlap': 64,
      'chunk_size': 512,
      'method': 'recursive'}],
    'embedding_models': ['ibm/slate-125m-english-rtrvr'],
    'generation': {'foundation_models': [{'model_id': 'mistralai/mistral-small-3-1-24b-instruct-2503'}]},
    'max_number_of_rag_patterns': 4,
    'retrieval_methods': ['simple']},
   'optimization': {'metrics': ['answer_correctness']},
   'output_logs': True},
  'results': [{'context': {'iteration': 0,
     'max_combinations': 2,
     'rag_pattern': {'composition_steps': ['model_selection',
       'chunking',
       'embeddings',
       'retrieval',
       'generation'],
      'duration_sec

### Get selected pattern

Get the RAGPattern object from the RAG Optimizer experiment. By default, the RAGPattern of the best pattern is returned.

In [19]:
best_pattern_name = summary.index.values[0]
print('Best pattern is:', best_pattern_name)

best_pattern = rag_optimizer.get_pattern(pattern_name="Pattern1")

Best pattern is: Pattern2


The pattern details can be retrieved by calling the `get_pattern_details` method:

```python
rag_optimizer.get_pattern_details(pattern_name='Pattern2')
```

### Create the index/collection

Build solution on the best pattern, with additional document indexing.

You can check which `index_name` you are working on:

```python
best_pattern.vector_store._index_name
```

In [20]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://ibm.github.io/watsonx-ai-python-sdk/fm_embeddings.html",
    "https://ibm.github.io/watsonx-ai-python-sdk/fm_custom_models.html",
    "https://ibm.github.io/watsonx-ai-python-sdk/fm_text_extraction.html"
]
docs_list = WebBaseLoader(urls).load()
doc_splits = best_pattern.chunker.split_documents(docs_list)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [21]:
best_pattern.indexing_function(doc_splits)

/var/folders/js/prfjw1gx7tqb554jl4sqw9pw0000gn/T/ipykernel_96767/1260819839.py:1: DeprecationWarning: `indexing_function` is deprecated and will be removed in a future release.
  best_pattern.indexing_function(doc_splits)


['8591131e50033fba603cbb386f8cdb553dfb1ce78e1ae5efebd66f6232b9ea8c',
 'c6ded0d72ad8bb1b2b31adb52e6de57f1e48a57d0103cfa5ade1714eb0b5ebd8',
 '32381dd4b918ae2c4bc88ce20bf314ce57fd8d94aeac88ac069a2b0f65d05dd9',
 '63bd9001dd1b0d3a96843ec90789cfa0539185e503b807cf7c4e9c810f0cf59a',
 '31d0e7b15a6d50e4074ba326bbc215adaafe1d417e93150ada3adf5d652fbb85',
 'bf834893766f79c474a698d7e74704e179929110b36c6d71f9290fc721470274',
 '73a6da22da234d2b58e62d9292ad0d28830110ecd4664716874c526a217f91f2',
 'ad36cf5048f49bae789428a05f3d6d12fb1c268fd01ac2047c562151d1ba0f47',
 '719535595c7836d36822bc567246061dc3c6ba4facb3acf46c0e7a9253feab3f',
 '5a9d36496ee5a13b95ee835cdbd2fc2b0b82635d86267e64a07973b42e780f3a',
 'b53ef0e8415975ac57f0cb6e63a10063528bc9394186dd0d0f81a27c774fdf71',
 '17ceba8a4fe063ec7751a5d2ebb33664f1128a7e5670da0a0df900bace3f98bd',
 '23386fbdbdfe78d502f274099641d9cbebf244c8134c0dbb9c87ed7a73f3ab53',
 '4d8c3685f7bae330b9ab35d3d592d3c86a68d7cd5805158d96354f16003644b3',
 '9a3ce2e2b6cd84b0da2cf497396f86cf

Query the RAGPattern locally, to test it.

In [22]:
from ibm_watsonx_ai.deployments import RuntimeContext

runtime_context = RuntimeContext(api_client=client)
inference_service_function = best_pattern.inference_service(runtime_context)[0]

In [25]:
question = "How to add Task Credentials?"

context = RuntimeContext(
    api_client=client,
    request_payload_json={"messages": [{"role": "user", "content": question}]},
)

inference_service_function(context)

{'body': {'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': ' </s>\n\nTo add Task Credentials, you need to follow these steps:\n\n**IBM watsonx.ai for IBM Cloud**\n\n1. **Create Credentials Object**: You can create a credentials object using either an API key or a token. Here are examples of both methods:\n\n   - **Using an API Key**:\n     ```python\n     from ibm_watsonx_ai import Credentials\n\n     credentials = Credentials(\n         url="https://us-south.ml.cloud.ibm.com",\n         api_key="YOUR_API_KEY"\n     )\n     ```\n\n   - **Using a Token**:\n     ```python\n     from ibm_watsonx_ai import Credentials\n\n     credentials = Credentials(\n         url="https://us-south.ml.cloud.ibm.com",\n         token="YOUR_TOKEN"\n     )\n     ```\n\n2. **Store Task Credentials**: Once you have the credentials object, you can store the task credentials using the `store` method:\n   ```python\n   client.task_credentials.store()\n   ```\n\n3. **List Available Ta

<a id="runs"></a>
## Historical runs

In this section you learn to work with historical RAG Optimizer jobs (runs).

To list historical runs use the `list()` method and provide the `'rag_optimizer'` filter.

In [27]:
experiment.runs(filter='rag_optimizer').list()

,timestamp,run_id,state,auto_pipeline_optimizer name
0,2025-06-26T07:21:42.823Z,b4b57086-916a-4228-af2d-e420159bcf08,completed,AutoAI RAG run - Base documentation


In [29]:
run_id = run_details['metadata']['id']
run_id

'b4b57086-916a-4228-af2d-e420159bcf08'

### Get executed optimizer's configuration parameters

In [30]:
experiment.runs.get_rag_params(run_id=run_id)

{'name': 'AutoAI RAG run - Base documentation',
 'description': 'AutoAI RAG Optimizer on ibm_watsonx_ai Base documentation',
 'chunking': [{'chunk_overlap': 64, 'chunk_size': 512, 'method': 'recursive'}],
 'embedding_models': ['ibm/slate-125m-english-rtrvr'],
 'retrieval_methods': ['simple'],
 'max_number_of_rag_patterns': 4,
 'generation': {'foundation_models': [{'model_id': 'mistralai/mistral-small-3-1-24b-instruct-2503'}]},
 'optimization_metrics': ['answer_correctness']}

### Get historical rag_optimizer instance and training details

In [31]:
historical_opt = experiment.runs.get_rag_optimizer(run_id)

### List trained patterns for selected optimizer

In [32]:
historical_opt.summary()

,mean_answer_correctness,mean_faithfulness,mean_context_correctness,chunking.method,chunking.chunk_size,chunking.chunk_overlap,embeddings.model_id,vector_store.distance_metric,retrieval.method,retrieval.number_of_chunks,retrieval.hybrid_ranker,generation.model_id
Pattern_Name,,,,,,,,,,,,
Pattern2,0.7083,0.2926,1.0,recursive,512,64,ibm/slate-125m-english-rtrvr,cosine,simple,5,,mistralai/mistral-small-3-1-24b-instruct-2503
Pattern1,0.4583,0.2808,1.0,recursive,512,64,ibm/slate-125m-english-rtrvr,cosine,simple,3,,mistralai/mistral-small-3-1-24b-instruct-2503


<a id="cleanup"></a>
## Clean up

To delete the current experiment, use the `cancel_run` method.

**Warning:** Be careful: once you delete an experiment, you will no longer be able to refer to it.

In [29]:
rag_optimizer.cancel_run()

'SUCCESS'

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to use `ibm-watsonx-ai` to run AutoAI RAG experiments. 

 Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Michał Steczko**, Software Engineer watsonx.ai

Copyright © 2024-2025 IBM. This notebook and its source code are released under the terms of the MIT License.